In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
df.head()

In [ ]:
# Clear value with missing value > 0
df = df.drop(columns=df.columns[df.isnull().sum() > 0])
# Remove the FLAG_DOCUMENT_* columns and SK_ID_CURR (since its represent the ID No. only)
column_save = list(filter(lambda x: not x.startswith('FLAG_DOCUMENT'), df.columns))
df = df[column_save].drop(columns='SK_ID_CURR')
df

In [ ]:
# Plot the target proportion
tg = df["TARGET"].value_counts()
plt.subplots(figsize=(12,8))
pie_target = tg.plot.pie(autopct="%.1f%%")
pie_target.legend(loc=1, labels={'Client with payment difficulties': 0, 'All other cases': 1})
plt.show()

In [ ]:
def dummy_variable_all(df, sort_frequence=True, dropna=True, map_show=False, skip_column=[]):
    for column_name in df:
        if np.dtype(df[column_name]) == 'O' and column_name not in skip_column:
            if sort_frequence:
                unique_value = df[column_name].value_counts(dropna=dropna).sort_values().index
            else:
                unique_value = df[column_name].value_counts(dropna=dropna).sort_index().index
            name_map = {}
            for i, value in enumerate(unique_value):
                name_map[value] = i
            if map_show:
                print('column_name :', column_name)
                print('replace :', name_map)
            df[column_name] = df[column_name].map(name_map)
    return df

In [ ]:
df2 = dummy_variable_all(df.copy(), dropna=False, map_show=True)
df2.fillna(df2.median(), inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)
df2

In [ ]:
df2.dtypes.value_counts()

In [ ]:
# Get the correlation
corr = df2.corr()

plt.subplots(figsize=(18, 12))
sns.heatmap(corr, linewidth=0.1)
plt.show()

In [ ]:
target_corr = abs(corr['TARGET'].copy().drop(index=['TARGET'])).sort_values(ascending=False)
target_corr

In [ ]:
# Extract the data
X = df2[df2.columns.copy().drop('TARGET')].values
y = df2['TARGET'].values
print(X.shape)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
X_new = SelectKBest(f_classif, k=2).fit_transform(X, y)
print(X_new)

In [ ]:
# Get the name of selected features
selected_features = []
for i in df2.columns:
    if all(df2.loc[:, i] == X_new[:, 0]) or all(df2.loc[:, i] == X_new[:, 1]):
        selected_features.append(i)
print(selected_features)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, train_size=0.75, test_size=0.25)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Normalized the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Compare before and after scaled
print(X_train)
print(X_train_scaled)
print(X_test)
print(X_test_scaled)

In [ ]:
#Create function to calculate all scores/evaluations
from sklearn.metrics import *

def evaluation(test, predict):
    acc_score = accuracy_score(test, predict)
    prec_score = precision_score(test, predict)
    rec_score = recall_score(test, predict)
    f_score = f1_score(test, predict)
    conf_matrix = confusion_matrix(test, predict)
    print('Accuracy : {:.3f}'.format(acc_score))
    print('Precision: {:.3f}'.format(prec_score))
    print('Recall   : {:.3f}'.format(rec_score))
    print('F        : {:.3f}'.format(f_score))
    print('Confusion matrix:\n', conf_matrix)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh5 = KNeighborsClassifier()
neigh5.fit(X_train_scaled, y_train)
predict_5 = neigh5.predict(X_test_scaled)
evaluation(y_test, predict_5)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train_scaled, y_train)
dt_predict = dt.predict(X_test_scaled)
evaluation(y_test, dt_predict)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)
rf_predict = rf.predict(X_test_scaled)
evaluation(y_test, rf_predict)

In [ ]:
knn5_prob = neigh5.predict_proba(X_test_scaled)
dt_prob = dt.predict_proba(X_test_scaled)
rf_prob = rf.predict_proba(X_test_scaled)

In [ ]:
average_prob = (knn5_prob + dt_prob + rf_prob) / 3
average_prob

In [ ]:
# Predict with application_train.csv
df_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
df_test

In [ ]:
X_test_file = df_test[selected_features].values

In [ ]:
# Normalized the data
X_test_file_scaled = scaler.transform(X_test_file)
X_test_file_scaled

In [ ]:
knn5_prob = neigh5.predict_proba(X_test_file_scaled)
dt_prob = dt.predict_proba(X_test_file_scaled)
rf_prob = rf.predict_proba(X_test_file_scaled)
average_prob = (knn5_prob + dt_prob + rf_prob) / 3
predict_res = 1 - average_prob[:, 0]

In [ ]:
pred = pd.Series(predict_res, name='TARGET')
final = pd.concat([df_test['SK_ID_CURR'], pred], axis=1)
final.shape

In [ ]:
import os
final.to_csv(os.path.join('','oof_all_01.csv'), index=False)